In [1]:
# import the required libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk
nltk.download('stopwords')
import re
import scipy
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec, KeyedVectors, FastText
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Flatten, SimpleRNN, RNN,GRU, SpatialDropout1D, Dropout
from keras.utils import pad_sequences



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vempa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Get the Data 

In [3]:
data_amz = pd.read_csv('amz_com-ecommerce_sample.csv', encoding= 'unicode_escape')
data_amz.shape 

data_flp = pd.read_csv('flipkart_com-ecommerce_sample.csv', encoding= 'unicode_escape')
data_flp.shape 


(20000, 15)

In [4]:
#check the amazon data
data_amz.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,982,438,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32143,29121,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati..."
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,991,551,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,694,325,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,208,258,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",..."


In [5]:
# check flipkart data
data_flp.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,999.0,379.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32157.0,22646.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati..."
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,699.0,267.0,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,220.0,210.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",..."


In [6]:
# tried to use product name and description at once but it didnt make any difference in the prediction so removed it

In [7]:
# data_amz['combined'] = data_amz["product_name"] + data_amz["description"]

In [8]:
# data_flp['combined'] = data_flp["product_name"] + data_flp["description"]

In [9]:
# data_flp['combined']

In [10]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vempa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [11]:
# # tokenization
tokens1 = []
tokens2 = []
tokens1 = [word_tokenize(str(sentence)) for sentence in data_amz['product_name']]
tokens2 = [word_tokenize(str(sentence)) for sentence in data_flp['product_name']]

In [13]:
#Removing whitespaces    
for i in tokens1:
    while '' in i:
        i.remove('')

for i in tokens2:
    while '' in i:
        i.remove('')
        
# Lowercasing
low1 = []
for i in tokens1:
    i = [x.lower() for x in i]
    low1.append(i)

low2 = []
for i in tokens2:
    i = [x.lower() for x in i]
    low2.append(i)

In [14]:
# Removing Stopwords
filter_words1 = []
Stopwords = set(stopwords.words('english'))

for word in tokens1:
    tokens3 = [w for w in word if w not in Stopwords]
    filter_words1.append(tokens3)
    
filter_words2 = []
for word in tokens2:
    tokens4 = [w for w in word if w not in Stopwords]
    filter_words2.append(tokens4)

### Word2Vec - CBOW

In [16]:
# Create CBOW model for each file separately
# after many iterations vector_size 135 looked like a better one
model1 = Word2Vec(filter_words1, min_count = 1,
                              vector_size = 135, window = 5)

model2 = Word2Vec(filter_words2, min_count = 1,
                              vector_size = 135, window = 5)



In [17]:
# create vectors
result = []
for sent1, sent2 in zip(filter_words1,filter_words2):
#     pdb.set_trace()

    vector1 = np.mean([model1.wv[word] for word in sent1], axis = 0)
    vector2 = np.mean([model2.wv[word] for word in sent2], axis = 0)
    cosine = scipy.spatial.distance.cosine(vector1, vector2)
    result.append((1-cosine)*100)
    
data_amz['Result[%]'] = result

In [18]:
subsetDataFrame = data_amz[data_amz['Result[%]'] > 50]
subsetDataFrame

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications,Result[%]
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,982,438,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ...",51.246345
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32143,29121,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati...",54.578203
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,991,551,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""...",61.900705
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,694,325,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ...",51.246345
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,208,258,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",...",52.721053
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19990,5847c8e9547dd50843f9d89886848ab9,2015-12-01 10:15:43 +0000,http://www.flipkart.com/elite-collection-mediu...,ELITE COLLECTION MEDIUM ACRYLIC STICKER,"[""Baby Care >> Baby & Kids Gifts >> Stickers >...",STIE88KNKXY33QZJ,1289,1293,"[""http://img6a.flixcart.com/image/sticker/q/z/...",False,Buy Elite Collection Medium Acrylic Sticker fo...,No rating available,No rating available,Elite Collection,"{""product_specification""=>[{""key""=>""Number of ...",52.277070
19993,a38fc11b1f6dc2d38bf38fae03ab6743,2015-12-01 10:15:43 +0000,http://www.flipkart.com/elite-collection-mediu...,ELITE COLLECTION MEDIUM ACRYLIC STICKER,"[""Baby Care >> Baby & Kids Gifts >> Stickers >...",STIE88KNYM5PHFJW,1282,1164,"[""http://img6a.flixcart.com/image/sticker/f/j/...",False,Buy Elite Collection Medium Acrylic Sticker fo...,No rating available,No rating available,Elite Collection,"{""product_specification""=>[{""key""=>""Number of ...",52.277070
19997,93e9d343837400ce0d7980874ece471c,2015-12-01 10:15:43 +0000,http://www.flipkart.com/elite-collection-mediu...,ELITE COLLECTION MEDIUM ACRYLIC STICKER,"[""Baby Care >> Baby & Kids Gifts >> Stickers >...",STIE7VAYDKQZEBSD,1284,1196,"[""http://img5a.flixcart.com/image/sticker/b/s/...",False,Buy Elite Collection Medium Acrylic Sticker fo...,No rating available,No rating available,Elite Collection,"{""product_specification""=>[{""key""=>""Number of ...",52.277070
19998,669e79b8fa5d9ae020841c0c97d5e935,2015-12-01 10:15:43 +0000,http://www.flipkart.c

In [19]:
# concat dataframes for final display of product name and price details
data_final_w2v_cbow = pd.concat([data_amz['product_name'], data_amz['retail_price'], data_amz['discounted_price'], data_flp['product_name'], data_flp['retail_price'], data_flp['discounted_price'], data_amz['Result[%]']], axis=1, ignore_index=False) 

In [20]:
data_final_w2v_cbow

,product_name,retail_price,discounted_price,product_name,retail_price,discounted_price,Result[%]
0,Alisha Solid Women's Cycling Shorts,982,438,Alisha Solid Women's Cycling Shorts,999.0,379.0,51.246345
1,FabHomeDecor Fabric Double Sofa Bed,32143,29121,FabHomeDecor Fabric Double Sofa Bed,32157.0,22646.0,54.578203
2,AW Bellies,991,551,AW Bellies,999.0,499.0,61.900705
3,Alisha Solid Women's Cycling Shorts,694,325,Alisha Solid Women's Cycling Shorts,699.0,267.0,51.246345
4,Sicons All Purpose Arnica Dog Shampoo,208,258,Sicons All Purpose Arnica Dog Shampoo,220.0,210.0,52.721053
...,...,...,...,...,...,...,...
19995,WALLDESIGN SMALL VINYL STICKER,1498,876,WallDesign Small Vinyl Sticker,1500.0,730.0,44.152412
19996,WALLMANTRA LARGE VINYL STICKERS STICKER,1415,1424,Wallmantra Large Vinyl Stickers Sticker,1429.0,1143.0,45.007521
19997,ELITE COLLECTION MEDIUM ACRYLIC STICKER,1284,1196,Elite Collection Medium Acrylic Sticker,1299.0,999.0,52.277070
19998,ELITE COLLECTION MEDIUM ACRYLIC STICKER,1492,1364,Elite Collection Medium Acrylic Sticker,1499.0,1199.0,52.277070


In [21]:
data_final_w2v_cbow[data_final_w2v_cbow['Result[%]']>50].drop(['Result[%]'], axis=1)

,product_name,retail_price,discounted_price,product_name,retail_price,discounted_price
0,Alisha Solid Women's Cycling Shorts,982,438,Alisha Solid Women's Cycling Shorts,999.0,379.0
1,FabHomeDecor Fabric Double Sofa Bed,32143,29121,FabHomeDecor Fabric Double Sofa Bed,32157.0,22646.0
2,AW Bellies,991,551,AW Bellies,999.0,499.0
3,Alisha Solid Women's Cycling Shorts,694,325,Alisha Solid Women's Cycling Shorts,699.0,267.0
4,Sicons All Purpose Arnica Dog Shampoo,208,258,Sicons All Purpose Arnica Dog Shampoo,220.0,210.0
...,...,...,...,...,...,...
19990,ELITE COLLECTION MEDIUM ACRYLIC STICKER,1289,1293,Elite Collection Medium Acrylic Sticker,1299.0,999.0
19993,ELITE COLLECTION MEDIUM ACRYLIC STICKER,1282,1164,Elite Collection Medium Acrylic Sticker,1299.0,999.0
19997,ELITE COLLECTION MEDIUM ACRYLIC STICKER,1284,1196,Elite Collection Medium Acrylic Sticker,1299.0,999.0
19998,ELITE COLLECTION MEDIUM ACRYLIC STICKER,1492,1364,Elite Collection Medium Acrylic Sticker,1499.0,1199.0


### Word2Vec - Skip Gram model

In [22]:
model3 = Word2Vec(filter_words1, min_count = 1, vector_size = 220,
                                             window = 5, sg = 1)

model4 = Word2Vec(filter_words2, min_count = 1, vector_size = 220,
                                             window = 5, sg = 1)


In [23]:
# import pdb
result = []
for sent1, sent2 in zip(filter_words1,filter_words2):
#     pdb.set_trace()
#     if word in model1.wv:
    vector1 = np.mean([model3.wv[word] for word in sent1], axis = 0)
    vector2 = np.mean([model4.wv[word] for word in sent2], axis = 0)
#     else:
#         pass
    cosine = scipy.spatial.distance.cosine(vector1, vector2)
    result.append((1-cosine)*100)
    
data_amz['Result[%]'] = result

In [24]:
subsetDataFrame = data_amz[data_amz['Result[%]'] > 50]
subsetDataFrame

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications,Result[%]
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,982,438,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ...",56.620795
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,991,551,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""...",52.759379
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,694,325,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ...",56.620795
6,ce5a6818f7707e2cb61fdcdbba61f5ad,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FVVKRBAXHB,1198,602,"[""http://img6a.flixcart.com/image/short/p/j/z/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ...",56.620795
9,4044c0ac52c1ee4b28777417651faf42,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FVUHAAVH9X,1197,542,"[""http://img5a.flixcart.com/image/short/5/z/c/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ...",56.620795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16603,0f6ffc643fe578f610e562fc00f1464e,2016-06-13 19:06:01 +0000,http://www.flipkart.com/wake-up-competition-fu...,Wake Up Competition Full Sleeve Solid Men's Sw...,"[""Clothing >> Men's Clothing >> Winter & Seaso...",SWSEJFF7HZU5ZCQZ,1481,879,"[""http://img5a.flixcart.com/image/sweatshirt/c...",False,Key Features of Wake Up Competition Full Sleev...,No rating available,No rating available,Wake Up Competition,"{""product_specification""=>[{""key""=>""Hooded"", ""...",55.275750
16604,4925cd62bb5f1501264c40973f8fa3d5,2016-06-13 19:06:01 +0000,http://www.flipkart.com/baalaa-printed-women-s...,Baalaa Printed Women's Straight Kurta,"[""Clothing >> Women's Clothing >> Ethnic Wear ...",KTAEJJCJSRQGZMZR,993,575,"[""http://img5a.flixcart.com/image/kurta/e/n/s/...",False,Key Features of Baalaa Printed Women's Straigh...,No rating available,No rating available,Baalaa,"{""product_specification""=>[{""key""=>""Sleeve"", ""...",62.340212
18223,c44bfa2dd5ebf6195138c56f67c86b66,2016-06-16 08:59:18 +0000,http://www.flipkart.com/fausto-sneakers/p/itme...,FAUSTO SNEAKERS,"[""Footwear >> Men's Footwear >> Casual Shoes >...",SHOEJP2GGNVTPHET,480,572,"[""http://img5a.flixcart.com/image/shoe/q/9/t/b...",False,Key Features of FAUSTO Sneakers Material: Synt...,No rating available,No rating available,FAUSTO,"{""product_specification""=>[{""key""=>""Ideal For""...",50.536531
18543,ebdc720c3ffffbddd7c93464f9731293,2016-06-14 18:49:44 +0000,http://www.flipkart.com/uncle-benit-monkey-cap...,UNCLE BENIT MON

In [25]:
# concat dataframes
data_final_w2v_sg = pd.concat([data_amz['product_name'], data_amz['retail_price'], data_amz['discounted_price'], data_flp['product_name'], data_flp['retail_price'], data_flp['discounted_price'], data_amz['Result[%]']], axis=1, ignore_index=False) 

In [26]:
data_final_w2v_sg

,product_name,retail_price,discounted_price,product_name,retail_price,discounted_price,Result[%]
0,Alisha Solid Women's Cycling Shorts,982,438,Alisha Solid Women's Cycling Shorts,999.0,379.0,56.620795
1,FabHomeDecor Fabric Double Sofa Bed,32143,29121,FabHomeDecor Fabric Double Sofa Bed,32157.0,22646.0,46.062830
2,AW Bellies,991,551,AW Bellies,999.0,499.0,52.759379
3,Alisha Solid Women's Cycling Shorts,694,325,Alisha Solid Women's Cycling Shorts,699.0,267.0,56.620795
4,Sicons All Purpose Arnica Dog Shampoo,208,258,Sicons All Purpose Arnica Dog Shampoo,220.0,210.0,41.430148
...,...,...,...,...,...,...,...
19995,WALLDESIGN SMALL VINYL STICKER,1498,876,WallDesign Small Vinyl Sticker,1500.0,730.0,17.665520
19996,WALLMANTRA LARGE VINYL STICKERS STICKER,1415,1424,Wallmantra Large Vinyl Stickers Sticker,1429.0,1143.0,17.484471
19997,ELITE COLLECTION MEDIUM ACRYLIC STICKER,1284,1196,Elite Collection Medium Acrylic Sticker,1299.0,999.0,18.420516
19998,ELITE COLLECTION MEDIUM ACRYLIC STICKER,1492,1364,Elite Collection Medium Acrylic Sticker,1499.0,1199.0,18.420516


In [27]:
data_final_w2v_sg[data_final_w2v_sg['Result[%]']>50].drop(['Result[%]'], axis=1)

,product_name,retail_price,discounted_price,product_name,retail_price,discounted_price
0,Alisha Solid Women's Cycling Shorts,982,438,Alisha Solid Women's Cycling Shorts,999.0,379.0
2,AW Bellies,991,551,AW Bellies,999.0,499.0
3,Alisha Solid Women's Cycling Shorts,694,325,Alisha Solid Women's Cycling Shorts,699.0,267.0
6,Alisha Solid Women's Cycling Shorts,1198,602,Alisha Solid Women's Cycling Shorts,1199.0,479.0
9,Alisha Solid Women's Cycling Shorts,1197,542,Alisha Solid Women's Cycling Shorts,1199.0,479.0
...,...,...,...,...,...,...
16603,Wake Up Competition Full Sleeve Solid Men's Sw...,1481,879,Wake Up Competition Full Sleeve Solid Men's Sw...,1495.0,748.0
16604,Baalaa Printed Women's Straight Kurta,993,575,Baalaa Printed Women's Straight Kurta,999.0,474.0
18223,FAUSTO SNEAKERS,480,572,FAUSTO Sneakers,499.0,499.0
18543,UNCLE BENIT MONKEY CAP CAP,681,409,UNCLE BENIT MONKEY CAP Cap,699.0,349.0


### Fast Text - Skip Gram model

In [28]:
ft_model1 = FastText(filter_words1,
                    max_n=0,
                    vector_size=300,
                    window=5,
                    min_count=1,
                    sg=1)
#                     workers=20,
#                     epochs=50,
#                     seed=42)

ft_model2 = FastText(filter_words2,
                    max_n=0,
                    vector_size=300,
                    window=5,
                    min_count=1,
                    sg=1)
#                     workers=20,
#                     epochs=50,
#                     seed=42)


In [29]:
# import pdb
result = []
for sent1, sent2 in zip(filter_words1,filter_words2):
#     pdb.set_trace()
#     if word in model1.wv:
    vector1 = np.mean([ft_model1.wv[word] for word in sent1], axis = 0)
    vector2 = np.mean([ft_model2.wv[word] for word in sent2], axis = 0)
#     else:
#         pass
    cosine = scipy.spatial.distance.cosine(vector1, vector2)
    result.append((1-cosine)*100)
    
data_amz['Result[%]'] = result

data_amz['Result[%]']

0        52.351475
1        46.586007
2        53.318721
3        52.351475
4        41.866976
           ...    
19995    19.183271
19996    18.132967
19997    27.125606
19998    27.125606
19999    27.125606
Name: Result[%], Length: 20000, dtype: float64

In [30]:
subsetDataFrame = data_amz[data_amz['Result[%]'] > 50]
subsetDataFrame

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications,Result[%]
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,982,438,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ...",52.351475
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,991,551,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""...",53.318721
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,694,325,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ...",52.351475
5,c2a17313954882c1dba461863e98adf2,2016-03-25 22:59:23 +0000,http://www.flipkart.com/eternal-gandhi-super-s...,Eternal Gandhi Super Series Crystal Paper Weig...,"[""Eternal Gandhi Super Series Crystal Paper We...",PWTEB7H2E4KCYUE3,427,473,"[""http://img5a.flixcart.com/image/paper-weight...",False,Key Features of Eternal Gandhi Super Series Cr...,No rating available,No rating available,Eternal Gandhi,"{""product_specification""=>[{""key""=>""Model Name...",50.283170
6,ce5a6818f7707e2cb61fdcdbba61f5ad,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FVVKRBAXHB,1198,602,"[""http://img6a.flixcart.com/image/short/p/j/z/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ...",52.351475
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19166,4f02c82e09375dcd8f11ab7cb16f5be1,2016-03-03 06:06:42 +0000,http://www.flipkart.com/camt-leather-documents...,CAMT LEATHER DOCUMENTS FILE,"[""Pens & Stationery >> Office Supplies >> File...",FAFEGERRHKYQWVKE,494,302,"[""http://img6a.flixcart.com/image/file-folder/...",False,Key Features of Camt Leather Documents File Fi...,No rating available,No rating available,Camt,"{""product_specification""=>[{""key""=>""Build Mate...",52.620661
19206,5510f63b552eba51de803852dd1f8d34,2016-03-03 06:06:42 +0000,http://www.flipkart.com/eaglemoss-publications...,EAGLEMOSS PUBLICATIONS DC SUPERHERO CHESS & MA...,"[""Toys & School Supplies >> Action Figures >> ...",AFGEFJSDRYS9ETH2,4254,4140,"[""http://img5a.flixcart.com/image/action-figur...",False,Specifications of Eaglemoss Publications Dc Su...,No rating available,No rating available,Eaglemoss Publications,"{""product_specification""=>{""key""=>""Character"",...",50.323468
19322,b7c2d5c9772f4b9fa944d33e23292447,2016-03-03 06:06:42 +0000,http://www.flipkart.com/whirlpool-affresh-cook...,WHIRLPOOL AFFRESH COOKTOP CLEANER KITCHEN CLEANER,"[""Household Supplies >> Home Cleaning >> Kitch...",KSCEG7CHTWWW4SPG,579,711,"[""http://img6a.flixcart.com/image/kitchen-surf...",False,Key Features of Whirlpool Affresh Cooktop Clea...,No rating available,No rating available,Whirlpool,"{""product_specification""=>[{""key""=>""Brand"", ""v...",51.543039
19396,da803ebdf3b6f79f4643c5d47e237227,2016-03-03 06:06:42 +0000,http://www.flipka

In [31]:
# concat dataframes
data_final_ft_sg = pd.concat([data_amz['product_name'], data_amz['retail_price'], data_amz['discounted_price'], data_flp['product_name'], data_flp['retail_price'], data_flp['discounted_price'], data_amz['Result[%]']], axis=1, ignore_index=False) 

In [32]:
data_final_ft_sg

,product_name,retail_price,discounted_price,product_name,retail_price,discounted_price,Result[%]
0,Alisha Solid Women's Cycling Shorts,982,438,Alisha Solid Women's Cycling Shorts,999.0,379.0,52.351475
1,FabHomeDecor Fabric Double Sofa Bed,32143,29121,FabHomeDecor Fabric Double Sofa Bed,32157.0,22646.0,46.586007
2,AW Bellies,991,551,AW Bellies,999.0,499.0,53.318721
3,Alisha Solid Women's Cycling Shorts,694,325,Alisha Solid Women's Cycling Shorts,699.0,267.0,52.351475
4,Sicons All Purpose Arnica Dog Shampoo,208,258,Sicons All Purpose Arnica Dog Shampoo,220.0,210.0,41.866976
...,...,...,...,...,...,...,...
19995,WALLDESIGN SMALL VINYL STICKER,1498,876,WallDesign Small Vinyl Sticker,1500.0,730.0,19.183271
19996,WALLMANTRA LARGE VINYL STICKERS STICKER,1415,1424,Wallmantra Large Vinyl Stickers Sticker,1429.0,1143.0,18.132967
19997,ELITE COLLECTION MEDIUM ACRYLIC STICKER,1284,1196,Elite Collection Medium Acrylic Sticker,1299.0,999.0,27.125606
19998,ELITE COLLECTION MEDIUM ACRYLIC STICKER,1492,1364,Elite Collection Medium Acrylic Sticker,1499.0,1199.0,27.125606


In [33]:
data_final_ft_sg[data_final_ft_sg['Result[%]']>50].drop(['Result[%]'], axis=1)

,product_name,retail_price,discounted_price,product_name,retail_price,discounted_price
0,Alisha Solid Women's Cycling Shorts,982,438,Alisha Solid Women's Cycling Shorts,999.0,379.0
2,AW Bellies,991,551,AW Bellies,999.0,499.0
3,Alisha Solid Women's Cycling Shorts,694,325,Alisha Solid Women's Cycling Shorts,699.0,267.0
5,Eternal Gandhi Super Series Crystal Paper Weig...,427,473,Eternal Gandhi Super Series Crystal Paper Weig...,430.0,430.0
6,Alisha Solid Women's Cycling Shorts,1198,602,Alisha Solid Women's Cycling Shorts,1199.0,479.0
...,...,...,...,...,...,...
19166,CAMT LEATHER DOCUMENTS FILE,494,302,Camt Leather Documents File,499.0,270.0
19206,EAGLEMOSS PUBLICATIONS DC SUPERHERO CHESS & MA...,4254,4140,Eaglemoss Publications Dc Superhero Chess & Ma...,4262.0,3622.0
19322,WHIRLPOOL AFFRESH COOKTOP CLEANER KITCHEN CLEANER,579,711,Whirlpool Affresh Cooktop Cleaner Kitchen Cleaner,599.0,599.0
19396,PLUS VALUE PVI- 052 REGULAR SQUARE CRYSTAL PEB...,435,528,Plus Value PVI- 052 Regular Square Crystal Peb...,450.0,450.0


### It looks like the Word2Vec CBOW model produced better results